This notebook allows the user to select XML collections and zip them up to send to a service that runs a transform on them and returns a simple CSV made up of seven data points. The data included is the Collection name, Dialect name, Record name, Concept name, Content, Xpath location, and the Dialect Definition for the concept. 

The notebook utilizes Bash and Python with the default packages contained in the Mac build of Anaconda with Python 3.6. 

Saxon, Java, and XSLT form the evaluation web service on a NCEAS virtual machine. 

This CSV contains a row for each concept that is found, so some elements may fulfill multiple concepts. A good example of this are the concepts Keyword and Place Keyword. Every Place Keyword is also a Keyword, so the row would repeat with a different Concept name. It also contains a row for each undefined node that contains text, marking these rows with an Unknown in the Concept column. 

This data can be used in a variety of analyses including RAD and QuickE as well as Concept Verticals. It can also be used to teach the system dialect definitions for concepts that are currently unknown by exposing all of the content at undefined nodes. 

## First we need to call all of the libraries we need to perform in our metadata wrangle

In [16]:
import pandas as pd
import os
from os import walk
import shutil
from ipywidgets import *
import ipywidgets as widgets
import requests
from contextlib import closing
import csv
import io

### Now let's select some metadata. 

If you have prepared metadata\* on your computer that you want to add, it is possible to upload into the repository locally using the [Add Metadata](AddMetadata.ipynb) Notebook before completing the following cells in this notebook. Otherwise, follow along and use some of the sample metadata the following steps will help you to select.

\* Prepared metadata contains a root element that has a standardized namespace and namespace prefix. Many dialects such as ISO and DIF are consistently written this way, but some dialects such as CSDGM are often written by organizations as only well-formed XML.

Create a list of subdirectories in the collection directory of MILE2 to select metadata for evaluation

In [17]:
Organizations = []
for (dirpath, dirnames, filenames) in walk('../collection/'):
    Organizations.extend(dirnames)
    break  

Create a function to select the organization the metadata comes from

In [18]:
def OrganizationChoices(organization):
    global OrganizationChoice
    global Organization
    Organization=organization
    print("Organization of the collection is", Organization)


Create a dropdown using the Organizations list and the organization selector function. This sets the Organization variable.

In [19]:
interactive(OrganizationChoices, organization=Organizations)

Create a list of collections in the organization directory selected in the dropdown above

In [20]:
Collections = []
for (dirpath, dirnames, filenames) in walk(os.path.join('../collection',Organization)):
    Collections.extend(dirnames)
    break 
Collections

['LTER_2005',
 'LTER_2006',
 'LTER_2007',
 'LTER_2008',
 'LTER_2009',
 'LTER_2010',
 'LTER_2011',
 'LTER_2012',
 'LTER_2013',
 'LTER_2014',
 'LTER_2015',
 'LTER_2016']

Create a function to select the collection the metadata comes from

In [21]:
def CollectionChoices(collection):
    global CollectionChoice
    global Collection
    Collection=collection

Create a dropdown using the Collections list and the organization selector function. This sets the Collection variable.

In [22]:
interactive(CollectionChoices, collection=Collections)

Many organizations support multiple metadata dialects, and share their collections in more than one dialect. This list is created the same way the others are. It adds the different dialects the collection is shared in to a list.

In [23]:
Dialects = []
for (dirpath, dirnames, filenames) in walk(os.path.join('../collection',Organization,Collection)):
    Dialects.extend(dirnames)
    break 
dialectList=Dialects


Create a function to select the dialect you want to send to the evaluator service.

In [24]:
def dialectChoice(dialect):
    global Dialect
    Dialect=dialect
    print("Dialect of the collection is", Dialect)


Create a dropdown using the Dialects list and the dialect selector function. This sets the Dialect variable.

In [25]:
interactive(dialectChoice,dialect=dialectList)

change to the zip directory 

In [26]:
%cd ../zip

/Users/scgordon/MILE2/zip


Combine the Organization, Collection, and Dialect variables with the string 'xml' as a relative path and save the string to a variable

In [27]:
MetadataDestination=os.path.join(Organization,Collection,Dialect,'xml')
MetadataDestination

'LTERthroughTime/LTER_2013/EML/xml'

Use the path to create a directory structure in the zip directory

In [28]:
os.makedirs(MetadataDestination, exist_ok=True)

Create a path to the metadata you selected earlier and save the string to a variable, 'MetadataLocation'.

In [29]:
MetadataLocation=os.path.join('../collection/',Organization,Collection,Dialect,'xml')

MetadataLocation

'../collection/LTERthroughTime/LTER_2013/EML/xml'

Copy the metadata to the new directory structure.

In [30]:
src_files = os.listdir(MetadataLocation)
for file_name in src_files:
    full_file_name = os.path.join(MetadataLocation, file_name)
    if (os.path.isfile(full_file_name)):
        shutil.copy(full_file_name, MetadataDestination)

Make a zip file to upload to the evaluator service

In [31]:
shutil.make_archive('../upload/metadata', 'zip', os.getcwd())

'/Users/scgordon/MILE2/upload/metadata.zip'

In [32]:
cd ../upload

/Users/scgordon/MILE2/upload


Send metadata to the Evaluator. Get the responses with csv encoding. This step can take up to a minute and doesn't track progress, but a dataframe or an error message will be returned.

In [33]:
url = 'http://metadig.nceas.ucsb.edu/metadata/evaluator'
files = {'zipxml': open('metadata.zip', 'rb')}
r = requests.post(url, files=files)
r.raise_for_status()
CollectionConceptsDF = pd.read_csv(io.StringIO(r.text), quotechar='"')
CollectionConceptsDF

,Collection,Dialect,Record,Concept,Content,XPath,DialectDefinition
0,LTER_2013,EML,00065-metadata.xml,Unknown,knb-lter-fce.1142.2,/eml:eml/@packageId,Undefined
1,LTER_2013,EML,00065-metadata.xml,Unknown,knb,/eml:eml/@system,Undefined
2,LTER_2013,EML,00065-metadata.xml,Unknown,eml://ecoinformatics.org/eml-2.1.0 http://nis....,/eml:eml/@xsi:schemaLocation,Undefined
3,LTER_2013,EML,00065-metadata.xml,Security Constraints,"uid=FCE,o=lter,dc=ecoinformatics,dc=org read p...",/eml:eml/access,/eml:eml/access
4,LTER_2013,EML,00065-metadata.xml,Resource Access Constraints,"uid=FCE,o=lter,dc=ecoinformatics,dc=org read p...",/eml:eml/access,/eml:eml/access
5,LTER_2013,EML,00065-metadata.xml,Metadata Access Constraints,"uid=FCE,o=lter,dc=ecoinformatics,dc=org read p...",/eml:eml/access,/eml:eml/access
6,LTER_2013,EML,00065-metadata.xml,Metadata Use Constraints,"uid=FCE,o=lter,dc=ecoinformatics,dc=org read p...",/eml:eml/access,/eml:eml/access
7,LTER_2013,EML,00065-metadata.xml,Resource Status,"uid=FCE,o=lter,dc=ecoinformatics,dc=org read p...",/eml:eml/access,/eml:eml/access
8,LTER_2013,EML,00065-metadata.xml,Unknown,PHY_Price_001,/eml:eml/dataset/@id,Undefined
9,LTER_2013,EML,00065-metadata.xml,Unknown,PHY_Price_001,/eml:eml/dataset/alternateIdentifier,Undefined


Save the dataframe as a csv for further analysis

In [23]:
CollectionConceptsDF.to_csv('../data/data.csv', mode = 'w', index=False)

Clear up temporary files and directories, switch to the data directory

In [24]:
%cd ../
shutil.rmtree('upload')
%cd zip
shutil.rmtree(Organization)
%cd ../data

/Users/scgordon/MILE2
/Users/scgordon/MILE2/zip
/Users/scgordon/MILE2/data


Copy the csv to a directory, named for the organization that had the metadata in it's holdings. Give it a filename matching the the metadata collection and dialect

In [25]:
shutil.copy("data.csv", os.path.join(Organization,Collection+'_'+Dialect+'_'+'data.csv'))

'NASA/GHRC_CSDGM_data.csv'

Now that we have our metadata data prepared and stored, we can look at recommendation analytics, concept content consistency, cross collection analytics, and help define unknown concepts.

### Select the notebook that prepares the data for different types of analysis

* [Create RAD Data](CreateRADdata.ipynb)
* [Concept Content Consistency](ConceptVerticals.ipynb)
* [Cross Collection Comparisons](QuickEvaluation-CrossCollectionComparisons.ipynb)
* [Exploring Unknown Concepts](ExploringUnknownConcepts.ipynb)